In [104]:
import pandas as pd 
import numpy as np
import math

In [7]:
pwd #placed the tsv file on the desktop

'/home/kristine/Desktop'

In [39]:
murine_v = pd.read_csv('Vgene_seqs_complete_murine.tsv', sep='\t') #importing the file

In [40]:
#defining the constants

#Alpha chain
#>Alpha 
TRAV1_2_03 = "GQNIDQPTEMTATEGAIVQINCTYQTSGFNGLFWYQQHAGEAPTFLSYNVLDGLEEKGRFSSFLSRSKGYSYLLLKELQMKDSASYLCAVR"
TRAJ2_01 = "NTGGLSGKLTFGEGTQVTVIS"

#Beta chain
#>Beta
TRBD1 = "GTGG" 
TRBJ1_3_01= "SGNTLYFGEGSRLIVV"

In [69]:
#Creating all the fasta files. Also on my desktop

for i in range(len(murine_v)):
    file1 = open(murine_v.gene[i]+".fasta","w") 
    alpha = [">alpha \n", TRAV1_2_03+TRAJ2_01]
    space = ["\n"]
    beta = [">beta \n", murine_v.seq[i]+TRBD1+TRBJ1_3_01]  
    file1.writelines(alpha)
    file1.writelines(space)
    file1.writelines(beta)

    file1.close()

In [71]:
#moved all the fasta files into a folder on the desktop names "fasta"

In [78]:
#now i need to figure out/remember how to use LYRA in the terminal. 

    # command : conda env list
# lyra runs in python 2.7 - i have this installed in my python 2 envionment
    # command: source activate python2_etc

    
#lyra_model ./TRBV1*01.fasta -o TRBV1*01.pdb

#Error: File format problem in trying to open HMM file ./TRBV12-1*02.fasta.
#Format tag is '>alpha': unrecognized.
#Current H3 format is 'HMMER3/f'. Previous H2/H3 formats also supported.


In [105]:
#### it is not working #### 
#I do not know how to trouble shoot - this might be due to the fact that i did not install it myself 
#in the first place, that was done by Magnus. Therefore i am using the online tool



#the online tool is used and all but TRBV30 and 31*1 and 31*2 was modelled. 
#they were trimmed using Milenas Jupyter notebook. 


#now to the calculation of the RMSD:

In [41]:
# ---------------------------------- IMPORT OF PACKAGES -----------------------
from Bio.PDB import *
import numpy as np
from Bio.SVDSuperimposer import SVDSuperimposer
import os
import pandas as pd
from glob import glob

# ---------------------------------- REFERENCES -------------------------------
#Bio.PDB:
#Hamelryck, T., Manderick, B. (2003) PDB parser and structure class implemented in Python. Bioinformatics 19: 2308–2310

#SVD superimposer:
#Matrix computations, 2nd ed. Golub, G. & Van Loan, CF., The Johns Hopkins University Press, Baltimore, 1989

#http://pldserver1.biochem.queensu.ca/~rlc/work/pymol/

#LYRA, SCRWL, SKLEARN, HMMER, Random forest
# --------------------------------------------------------------------------------

#choose some true structures, PDB file format
parser = PDBParser()

def pdb_load(pdb_files):
    """a function that converts a pdb path into a strcutural file"""
    parser = PDBParser()
    
    structure_list = []
    
    for fileName in pdb_files:
        #the fileName is the whole path
        #structre_id is the created based on the path, cutting it at the last "/" and removing the last letters "_CDRX.pdb"
        structure_id = fileName.rsplit('/', 1)[1][:-9]
        #the stucture is imported
        structure = parser.get_structure(structure_id, fileName)
        
        structure_list.append(structure)
    
    return structure_list

def get_CA_coordinates(structure):
    """this function return a vector of CA atom coordinates"""
    #empty list
    x = []
    
    for model in structure.get_list():
        for chain in model.get_list():
            for residue in chain.get_list():
                if residue.has_id("CA"):
                    ca = residue["CA"]
                    a = ca.get_coord()
                    
                    x.append(a)
                    
    coordinates = np.array(x)
    return coordinates

def get_coordinates(structure_list):
    """a function that allow coordinate retraction of an entire list"""
    coordinate_list = []
    
    for ii in structure_list:
        coordinates = get_CA_coordinates(ii)
        
        coordinate_list.append(coordinates)
    return coordinate_list

def get_RMSD_vers2(structure1_coordinates, structure2_coordinates):
    """a function that calculate RMSD of two stuctures"""
    sup = SVDSuperimposer()
    
    if len(structure1_coordinates) != len(structure2_coordinates):
        
        if len(structure1_coordinates) < len(structure2_coordinates):
            
            length = len(structure1_coordinates)
 
            #creates two empy matrixes that is the length of the fixed. 

            structure1_coordinates_coord = np.zeros((length, 3))
            structure2_coordinates_coord = np.zeros((length, 3))
        
            for i in range(0, length): 
                structure1_coordinates_coord[i] = structure1_coordinates[i]
                structure2_coordinates_coord[i] = structure2_coordinates[i]

            sup.set(structure1_coordinates_coord, structure2_coordinates_coord)
            sup.run()
            rms = sup.get_rms()
            return("%.2f"%rms)
        
        elif len(structure2_coordinates) < len(structure1_coordinates):
            
            length = len(structure2_coordinates)
            #creates two empy matrixes that is the length of the fixed. 

            structure2_coordinates_coord = np.zeros((length, 3))
            structure1_coordinates_coord = np.zeros((length, 3))
        
            for i in range(0, length): 
                structure2_coordinates_coord[i] = structure2_coordinates[i]
                structure1_coordinates_coord[i] = structure1_coordinates[i]

            
            sup.set(structure2_coordinates_coord, structure1_coordinates_coord)
            sup.run()
            rms = sup.get_rms()
            return("%.2f"%rms)

    else:
        sup.set(structure1_coordinates,structure2_coordinates)
        sup.run()
        rmsd = sup.get_rms()
        return ("%.2f"%rmsd)

### get them into a matrix
        
def RMSD_metric(x,y,names):
    """a function that allow RMSD to be calculated on many"""
    matrix = []
    
    for i in x:
        for j in y:
            t = get_RMSD_vers2(i,j)
            matrix.append(t)
            
    matrix = np.array(matrix)
    matrix = np.reshape(matrix, (len(names), len(names)))

    #naming the columns and rows, and converting it to a data-frame. 
    df = pd.DataFrame(matrix)
    df.columns = (names)
    df.index = (names)
    return df 


def run_canonical_rmsd(pdb_files):
    """ a function that pull data through all the other functions"""
    names = []
    for ii in pdb_files:
        id = ii.rsplit('/', 1)[1][:-9]
        names.append(id)
    
    x = pdb_load(pdb_files) 
    x_coordinates = get_coordinates(x)
    RMSD = RMSD_metric(x_coordinates, x_coordinates, names)

    return RMSD

In [103]:
#### running it ####
def RMSD_dist_file(path_to_pdb_files, cdr_n):
    """path to pdb files to include last /.
    
    cdr_n to be either 'CDR1' or 'CDR2'
    """
    cdr_n_files = glob(path_to_pdb_files +'*'+cdr_n+'.pdb')
    
    cdr_n_RMSD = run_canonical_rmsd(cdr_n_files)
    cdr_n_max_rmsd = cdr_n_RMSD.max().max()
    
    alpha = cdr_n_RMSD.to_numpy()
    alpha = alpha.astype(float)
    alpha = np.divide(alpha, cdr_n_max_rmsd) 
    cdr_n_RMSD = pd.DataFrame(alpha,cdr_n_RMSD.index,cdr_n_RMSD.columns)
    
    return cdr_n_RMSD

CDR1_RMSD = RMSD_dist_file('/home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/','CDR1')
CDR1_RMSD.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/CDR1_RMSD.csv', index = True)

CDR2_RMSD = RMSD_dist_file('/home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/','CDR2')
CDR2_RMSD.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/CDR2_RMSD.csv', index = True)

/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'S' for Atom (name=SG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=C) with given element ''
  warnings.warn(msg, PDBConstructionWarnin

/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'S' for Atom (name=SG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=C) with given element ''
  warnings.warn(msg, PDBConstructionWarnin

/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CG) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CD1) with given element ''
  warnings.warn(msg, PDBConstructionWarn

/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CG2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CG1) with given element ''
  warnings.warn(msg, PDBConstructionWar

/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CG2) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/kristine/anaconda3/lib/python3.7/site-packages/Bio/PDB/Atom.py:189: PDBConstructionWarning: Used element 'C' for Atom (name=CG1) with given element ''
  warnings.warn(msg, PDBConstructionWar

In [107]:
################ the TM score. 

#The only module for TM score calculation i could find last fall did not work. 
#Therefore i downloaded the TM software from Zhang et al.

#lets see if i can get it to run

In [ ]:
########################### The problem

#ERROR! Undefined option /home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/TRBV12-2*01_CDR1.pdb
#ERROR! Undefined option /home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/TRBV21*01_CDR1.pdb

#The problem is, that TM scoring takes offense to the below for some reason

#TRBV12-2*01_CDR2.pdb
#TRBV12-2*02_CDR2.pdb
#TRBV21*01_CDR2.pdb


In [256]:
############# Lets do it for CDR2
CDR1_files = glob('/home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/*CDR1.pdb')
CDR2_files = glob('/home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/*CDR2.pdb')

#print("./TMscore "+cs1_pdb_files[0]+" "+cs1_pdb_files[1])
# i have an empty file called terminal_output.txt ready to print in


def doc1(liste,new,n):
    x = []
    for i in range(len(liste)):
        for j in range(len(new)):
            y = "./TMscore "+new[j]+" "+liste[i] + " >> terminal_output_CDR%s.txt" %n
            u = "echo %s,%s >> terminal_output_CDR%s.txt \n" % (i,j,n)
            x.append(y)
            x.append(u)
        #y = "./TMscore /home/kristine/Documents/Thesis/Thesis_Ubuntu/Canonical_structures/CDR1/TRBV5-6_01_CDR1.pdb "+liste[i]    
        #x.append(y)
    print(len(x))
    return x

TM_CDR1 = doc1(CDR1_files, CDR1_files, 1)
TM_CDR2 = doc1(CDR2_files, CDR2_files, 2)
#output:./TMscore /home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/TRBV3*02_CDR1.pdb /home/kristine/Desktop/Canonical_structures_generation/CDR1_CD2_loops/TRBV3*02_CDR1.pdb

#type(TM_CDR1)
with open('tm_commands_CDR1.txt', 'w') as f:
    for item in TM_CDR1:
        f.write("%s\n" % item)

with open('tm_commands_CDR2.txt', 'w') as f:
    for item in TM_CDR2:
        f.write("%s\n" % item)

#In shell: 
#./tm_commands_CDR1.sh
#./tm_commands_CDR2.sh

4232
4232


In [263]:
CDR1_names = []
for i in range(len(CDR1_files)):
    name = CDR1_files[i].rsplit('/', 1)[1][:-9]
    CDR1_names.append(name)

In [ ]:
###########################################

# cannot get it to work because this too takes offense to 


#TRBV12-2*01_CDR2.pdb
#TRBV12-2*02_CDR2.pdb
#TRBV21*01_CDR2.pdb

###########################################

In [264]:
def lines_that_contain(string, fp):
    return [line for line in fp if string in line]

def getting_tm(textfile):
    with open(textfile, "r") as fp:
        score = []        
        for line in lines_that_contain("TM-score    = ", fp):
            a = float(line[14:20]) 
            score.append(a)
    return score

    #comes out in the same ordxer. 

def reshape(path):   
    list_of_TM = getting_tm(path)
    
    b = list_of_TM
    b = np.array(b)
    b = np.reshape(b, (len(CDR1_names),len(CDR1_names))) #should be 46
    cdr_TM = pd.DataFrame(b, CDR1_names, CDR1_names)
    
    return cdr_TM
                   
                   
CDR1_TM = reshape("/home/kristine/Downloads/terminal_output_CDR1.txt")
CDR1_TM.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/CDR1_TM.csv', index = True)

CDR2_TM = reshape("/home/kristine/Downloads/terminal_output_CDR2.txt")
CDR2_TM.to_csv(r'/home/kristine/Desktop/Canonical_structures_generation/Libraries/CDR2_TM.csv', index = True)